In [1]:
#IMPORT BASIC LIBRARIES
import numpy as np
import pandas as pd
import statistics
import tensorflow as tf

# Importing Tensorflow Libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense, TimeDistributed
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.optimizers import Adam
from keras.initializers import Constant

#IMPORT NLP LIBRARY
import nltk
import string
import spacy
import pickle

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# Load your DataFrame
df = pd.read_excel('/content/restaurant_reviews-v2-1.xlsx')

# KEEP ONLY NECESSARY COLUMNS
df = df[['Review', 'positive=1/negative=0']]
df.head()

,Review,positive=1/negative=0
0,Great food and great atmosphere! The chicken t...,1
1,I had heard good things about Tikka Shak so I ...,0
2,I was driving by tikka shack one day and decid...,0
3,Tikka Shack had the most modern and up-to-date...,1
4,Today is the third time I've come to India Pal...,1


In [3]:
# PREPROCESSING: remove stopwords and punctuation
df = df.reset_index(drop=True)
df['reduced'] = ""
stopWords = set(stopwords.words('english'))
for index, row in df.iterrows():
    temp = row['Review']
    tokens_without_punct = nltk.word_tokenize(''.join(filter(lambda x: x not in string.punctuation, temp))) # removed punctuation
    removal = [word for word in tokens_without_punct if word.lower() not in stopWords] # removed stopwords
    result = ' '.join(removal)
    df.at[index,'reduced'] = result

# PREPROCESSING: perform lemmatization
nlp = spacy.load('en_core_web_sm')
df = df.reset_index(drop=True)
df['lemmatized'] = ""
for index, row in df.iterrows():
    temp = row['reduced']
    doc = nlp(temp)
    tokens = []
    for token in doc:
      tokens.append(token)
    lemma = " ".join([token.lemma_ for token in doc])
    df.at[index,'lemmatized'] = lemma

In [4]:
# Labeling using the RAW INPUT DATA
texts_raw = df["Review"] # Raw Input Data
labels = df['positive=1/negative=0'] # Output labels (binary)
# Tokenize the RAW text
tokenizer_raw = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer_raw.fit_on_texts(texts_raw)
sequences_raw = tokenizer_raw.texts_to_sequences(texts_raw)
# Pad the RAW sequences
padded_sequences_raw = pad_sequences(sequences_raw, maxlen=120, padding='post', truncating='post')


# Labeling the Preprocssed INPUT DATA
texts_prep = df["lemmatized"] # Preprocessed Input data
# labels = df['positive=1/negative=0']
# Tokenize the Preprocssed text
tokenizer_prep = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer_prep.fit_on_texts(texts_prep)
sequences_prep = tokenizer_prep.texts_to_sequences(texts_prep)
# Pad the Preprocssed sequences
padded_sequences_prep = pad_sequences(sequences_prep, maxlen=120, padding='post', truncating='post')

In [5]:
# TF-IDF Implementation for RAW INPUT DATA
# Convert texts to TF-IDF features
tfidf_vectorizer_raw = TfidfVectorizer(max_features=10000)  # We can adjust the number of features
# Save tfidf_vectorizer as a pickle file for RAW DATA
X_tfidf_raw = tfidf_vectorizer_raw.fit_transform(df['Review'])
pickle.dump(tfidf_vectorizer_raw, open('tf_uni_raw.pkl', 'wb'))
X_tfidf_raw = X_tfidf_raw.toarray()
# Split the data: TF-IDF
X_train_tfidf_raw, X_test_tfidf_raw, y_train_tfidf_raw, y_test_tfidf_raw = train_test_split(X_tfidf_raw, labels, test_size=0.2, random_state=42, stratify=labels)


# TF-IDF Implementation for Preprocessed DATA
# Convert texts to TF-IDF features
tfidf_vectorizer_prep = TfidfVectorizer(max_features=10000)  # We can adjust the number of features
# Save tfidf_vectorizer as a pickle file
X_tfidf_prep = tfidf_vectorizer_prep.fit_transform(df['lemmatized'])
pickle.dump(tfidf_vectorizer_prep, open('tf_uni_prep.pkl', 'wb'))
X_tfidf_prep = X_tfidf_prep.toarray()
# Split the data: TF-IDF
X_train_tfidf_prep, X_test_tfidf_prep, y_train_tfidf_prep, y_test_tfidf_prep = train_test_split(X_tfidf_prep, labels, test_size=0.2, random_state=42, stratify=labels)

In [6]:
# GloVe Implementation
# Load GloVe embeddings
glove_embeddings = {}
with open('/content/glove.twitter.27B.100d.txt', 'r', encoding='utf8') as f:  # change the files for different glove dimensions
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        glove_embeddings[word] = vector


# Creating an embedding matrix with RAW DATA
embedding_dim = 100  # This should match the GloVe embeddings dimension
vocab_size_raw = len(tokenizer_raw.word_index) + 1  # Adding 1 because of reserved 0 index
embedding_matrix_raw = np.zeros((vocab_size_raw, embedding_dim))
for word, i in tokenizer_raw.word_index.items():
    embedding_vector_raw = glove_embeddings.get(word)
    if embedding_vector_raw is not None:
        embedding_matrix_raw[i] = embedding_vector_raw
# Split the RAW data: GloVe
X_train_glove_raw, X_test_glove_raw, y_train_glove_raw, y_test_glove_raw = train_test_split(padded_sequences_raw, labels, test_size=0.2, random_state=42, stratify=labels)


# Creating an embedding matrix with Preprocssed Data
embedding_dim = 100  # This should match the GloVe embeddings dimension
vocab_size_prep = len(tokenizer_prep.word_index) + 1  # Adding 1 because of reserved 0 index
embedding_matrix_prep = np.zeros((vocab_size_prep, embedding_dim))
for word, i in tokenizer_prep.word_index.items():
    embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix_prep[i] = embedding_vector
# Split the Preprocssed data: GloVe
X_train_glove_prep, X_test_glove_prep, y_train_glove_prep, y_test_glove_prep = train_test_split(padded_sequences_prep, labels, test_size=0.2, random_state=42, stratify=labels)

In [7]:
# Model for TF-IDF - RAW - NO DROPOUT
birnn_tfidf_raw = tf.keras.Sequential([
    # Since TF-IDF vectors are not sequential data like word embeddings, we use a Dense network instead of RNN layers.
    tf.keras.layers.Dense(embedding_dim, input_shape=(X_train_tfidf_raw.shape[1],), activation='relu'),
    tf.keras.layers.Reshape((10, 10), input_shape=(X_train_tfidf_raw.shape[1],)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
birnn_tfidf_raw.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=3e-4), metrics=['accuracy'])
birnn_tfidf_raw.fit(X_train_tfidf_raw, y_train_tfidf_raw, epochs=20, validation_data=(X_test_tfidf_raw, y_test_tfidf_raw))

Epoch 1/20
7/7 [==============================] - 13s 533ms/step - loss: 0.6933 - accuracy: 0.5192 - val_loss: 0.6925 - val_accuracy: 0.5192
Epoch 2/20
7/7 [==============================] - 0s 62ms/step - loss: 0.6919 - accuracy: 0.5192 - val_loss: 0.6921 - val_accuracy: 0.5192
Epoch 3/20
7/7 [==============================] - 0s 61ms/step - loss: 0.6901 - accuracy: 0.5192 - val_loss: 0.6910 - val_accuracy: 0.5192
Epoch 4/20
7/7 [==============================] - 0s 63ms/step - loss: 0.6851 - accuracy: 0.5192 - val_loss: 0.6881 - val_accuracy: 0.5192
Epoch 5/20
7/7 [==============================] - 0s 55ms/step - loss: 0.6722 - accuracy: 0.6394 - val_loss: 0.6799 - val_accuracy: 0.5577
Epoch 6/20
7/7 [==============================] - 0s 61ms/step - loss: 0.6366 - accuracy: 0.8077 - val_loss: 0.6572 - val_accuracy: 0.5577
Epoch 7/20
7/7 [==============================] - 0s 55ms/step - loss: 0.5388 - accuracy: 0.9231 - val_loss: 0.5867 - val_accuracy: 0.6731
Epoch 8/20
7/7 [=========

In [8]:
# Model for TF-IDF - RAW - WITH DROPOUT
birnn_tfidf_raw_drop = tf.keras.Sequential([
    tf.keras.layers.Dense(embedding_dim, input_shape=(X_train_tfidf_raw.shape[1],), activation='relu'),
    tf.keras.layers.Reshape((10, 10), input_shape=(X_train_tfidf_raw.shape[1],)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim, return_sequences=True)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
birnn_tfidf_raw_drop.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=3e-4), metrics=['accuracy'])
birnn_tfidf_raw_drop.fit(X_train_tfidf_raw, y_train_tfidf_raw, epochs=20, validation_data=(X_test_tfidf_raw, y_test_tfidf_raw))

Epoch 1/20
7/7 [==============================] - 12s 371ms/step - loss: 0.6932 - accuracy: 0.5337 - val_loss: 0.6924 - val_accuracy: 0.5192
Epoch 2/20
7/7 [==============================] - 0s 64ms/step - loss: 0.6918 - accuracy: 0.5192 - val_loss: 0.6917 - val_accuracy: 0.5192
Epoch 3/20
7/7 [==============================] - 0s 68ms/step - loss: 0.6895 - accuracy: 0.5192 - val_loss: 0.6899 - val_accuracy: 0.5192
Epoch 4/20
7/7 [==============================] - 0s 59ms/step - loss: 0.6840 - accuracy: 0.5192 - val_loss: 0.6857 - val_accuracy: 0.5192
Epoch 5/20
7/7 [==============================] - 0s 60ms/step - loss: 0.6692 - accuracy: 0.5913 - val_loss: 0.6754 - val_accuracy: 0.5385
Epoch 6/20
7/7 [==============================] - 0s 61ms/step - loss: 0.6313 - accuracy: 0.7356 - val_loss: 0.6464 - val_accuracy: 0.5962
Epoch 7/20
7/7 [==============================] - 0s 65ms/step - loss: 0.5275 - accuracy: 0.9231 - val_loss: 0.5629 - val_accuracy: 0.7308
Epoch 8/20
7/7 [=========

In [9]:
# Model for TF-IDF - PREPROCESSED - NO DROPOUT
birnn_tfidf_prep = tf.keras.Sequential([
    tf.keras.layers.Dense(embedding_dim, input_shape=(X_train_tfidf_prep.shape[1],), activation='relu'),
    tf.keras.layers.Reshape((10, 10), input_shape=(X_train_tfidf_prep.shape[1],)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
birnn_tfidf_prep.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=3e-4), metrics=['accuracy'])
birnn_tfidf_prep.fit(X_train_tfidf_prep, y_train_tfidf_prep, epochs=20, validation_data=(X_test_tfidf_prep, y_test_tfidf_prep))

Epoch 1/20
7/7 [==============================] - 13s 381ms/step - loss: 0.6931 - accuracy: 0.4856 - val_loss: 0.6928 - val_accuracy: 0.5192
Epoch 2/20
7/7 [==============================] - 1s 109ms/step - loss: 0.6926 - accuracy: 0.5192 - val_loss: 0.6917 - val_accuracy: 0.5192
Epoch 3/20
7/7 [==============================] - 1s 102ms/step - loss: 0.6897 - accuracy: 0.5192 - val_loss: 0.6904 - val_accuracy: 0.5192
Epoch 4/20
7/7 [==============================] - 1s 104ms/step - loss: 0.6854 - accuracy: 0.5192 - val_loss: 0.6872 - val_accuracy: 0.5192
Epoch 5/20
7/7 [==============================] - 1s 107ms/step - loss: 0.6742 - accuracy: 0.5192 - val_loss: 0.6796 - val_accuracy: 0.5192
Epoch 6/20
7/7 [==============================] - 1s 101ms/step - loss: 0.6441 - accuracy: 0.5529 - val_loss: 0.6590 - val_accuracy: 0.5192
Epoch 7/20
7/7 [==============================] - 1s 104ms/step - loss: 0.5628 - accuracy: 0.8173 - val_loss: 0.5985 - val_accuracy: 0.7500
Epoch 8/20
7/7 [===

In [10]:
# Model for TF-IDF - PREPROCESSED - WITH DROPOUT
birnn_tfidf_prep_drop = tf.keras.Sequential([
    tf.keras.layers.Dense(embedding_dim, input_shape=(X_train_tfidf_prep.shape[1],), activation='relu'),
    tf.keras.layers.Reshape((10, 10), input_shape=(X_train_tfidf_prep.shape[1],)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim, return_sequences=True)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
birnn_tfidf_prep_drop.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=3e-4), metrics=['accuracy'])
birnn_tfidf_prep_drop.fit(X_train_tfidf_prep, y_train_tfidf_prep, epochs=20, validation_data=(X_test_tfidf_prep, y_test_tfidf_prep))

Epoch 1/20
7/7 [==============================] - 12s 478ms/step - loss: 0.6937 - accuracy: 0.3942 - val_loss: 0.6926 - val_accuracy: 0.5192
Epoch 2/20
7/7 [==============================] - 0s 64ms/step - loss: 0.6922 - accuracy: 0.5192 - val_loss: 0.6922 - val_accuracy: 0.5192
Epoch 3/20
7/7 [==============================] - 0s 64ms/step - loss: 0.6905 - accuracy: 0.5192 - val_loss: 0.6909 - val_accuracy: 0.5192
Epoch 4/20
7/7 [==============================] - 0s 56ms/step - loss: 0.6862 - accuracy: 0.5288 - val_loss: 0.6877 - val_accuracy: 0.5192
Epoch 5/20
7/7 [==============================] - 0s 56ms/step - loss: 0.6753 - accuracy: 0.5240 - val_loss: 0.6801 - val_accuracy: 0.5192
Epoch 6/20
7/7 [==============================] - 0s 55ms/step - loss: 0.6465 - accuracy: 0.5721 - val_loss: 0.6608 - val_accuracy: 0.5192
Epoch 7/20
7/7 [==============================] - 0s 60ms/step - loss: 0.5720 - accuracy: 0.8077 - val_loss: 0.6064 - val_accuracy: 0.6923
Epoch 8/20
7/7 [=========

In [11]:
# BiRNN model using GloVe embeddings - RAW DATA - NO DROPOUT
birnn_glove_raw = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size_raw, embedding_dim, input_length=X_train_glove_raw.shape[1], weights=[embedding_matrix_raw], trainable=False),
    # BiRNN model is similar to LSTM model but uses Bidirectional wrappers around the LSTM layers
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
birnn_glove_raw.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=3e-4), metrics=['accuracy'])
birnn_glove_raw.fit(X_train_glove_raw, y_train_glove_raw, epochs=20, validation_data=(X_test_glove_raw, y_test_glove_raw))

Epoch 1/20
7/7 [==============================] - 14s 739ms/step - loss: 0.7018 - accuracy: 0.4327 - val_loss: 0.6907 - val_accuracy: 0.5192
Epoch 2/20
7/7 [==============================] - 5s 720ms/step - loss: 0.6747 - accuracy: 0.6058 - val_loss: 0.6827 - val_accuracy: 0.5962
Epoch 3/20
7/7 [==============================] - 4s 549ms/step - loss: 0.6524 - accuracy: 0.6635 - val_loss: 0.6655 - val_accuracy: 0.6731
Epoch 4/20
7/7 [==============================] - 3s 478ms/step - loss: 0.6201 - accuracy: 0.6875 - val_loss: 0.6344 - val_accuracy: 0.6731
Epoch 5/20
7/7 [==============================] - 3s 475ms/step - loss: 0.5456 - accuracy: 0.8077 - val_loss: 0.5506 - val_accuracy: 0.7115
Epoch 6/20
7/7 [==============================] - 5s 771ms/step - loss: 0.4704 - accuracy: 0.8173 - val_loss: 0.5406 - val_accuracy: 0.7692
Epoch 7/20
7/7 [==============================] - 4s 485ms/step - loss: 0.4742 - accuracy: 0.8125 - val_loss: 0.6916 - val_accuracy: 0.6154
Epoch 8/20
7/7 [===

In [12]:
# BiRNN model using GloVe embeddings - RAW DATA - WITH DROPOUT
birnn_glove_raw_drop = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size_raw, embedding_dim, input_length=X_train_glove_raw.shape[1], weights=[embedding_matrix_raw], trainable=False),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim, return_sequences=True)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
birnn_glove_raw_drop.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=3e-4), metrics=['accuracy'])
birnn_glove_raw_drop.fit(X_train_glove_raw, y_train_glove_raw, epochs=20, validation_data=(X_test_glove_raw, y_test_glove_raw))

Epoch 1/20
7/7 [==============================] - 18s 1s/step - loss: 0.6918 - accuracy: 0.4760 - val_loss: 0.6786 - val_accuracy: 0.5769
Epoch 2/20
7/7 [==============================] - 4s 502ms/step - loss: 0.6636 - accuracy: 0.6731 - val_loss: 0.6639 - val_accuracy: 0.6731
Epoch 3/20
7/7 [==============================] - 3s 467ms/step - loss: 0.6357 - accuracy: 0.6490 - val_loss: 0.6364 - val_accuracy: 0.7115
Epoch 4/20
7/7 [==============================] - 3s 482ms/step - loss: 0.5918 - accuracy: 0.7740 - val_loss: 0.6082 - val_accuracy: 0.6731
Epoch 5/20
7/7 [==============================] - 6s 846ms/step - loss: 0.5142 - accuracy: 0.8221 - val_loss: 0.5350 - val_accuracy: 0.7692
Epoch 6/20
7/7 [==============================] - 3s 476ms/step - loss: 0.4447 - accuracy: 0.8077 - val_loss: 0.6976 - val_accuracy: 0.6346
Epoch 7/20
7/7 [==============================] - 3s 478ms/step - loss: 0.4628 - accuracy: 0.7933 - val_loss: 0.4722 - val_accuracy: 0.7885
Epoch 8/20
7/7 [======

In [13]:
# BiRNN model using GloVe embeddings - PREPROCESSED DATA - NO DROPOUT
birnn_glove_prep = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size_prep, embedding_dim, input_length=X_train_glove_prep.shape[1], weights=[embedding_matrix_prep], trainable=False),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
birnn_glove_prep.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=3e-4), metrics=['accuracy'])
birnn_glove_prep.fit(X_train_glove_prep, y_train_glove_prep, epochs=20, validation_data=(X_test_glove_prep, y_test_glove_prep))

Epoch 1/20
7/7 [==============================] - 16s 1s/step - loss: 0.6879 - accuracy: 0.5096 - val_loss: 0.6768 - val_accuracy: 0.6154
Epoch 2/20
7/7 [==============================] - 3s 467ms/step - loss: 0.6559 - accuracy: 0.6635 - val_loss: 0.6550 - val_accuracy: 0.6923
Epoch 3/20
7/7 [==============================] - 3s 472ms/step - loss: 0.6181 - accuracy: 0.7067 - val_loss: 0.6286 - val_accuracy: 0.6346
Epoch 4/20
7/7 [==============================] - 4s 633ms/step - loss: 0.5815 - accuracy: 0.7356 - val_loss: 0.5489 - val_accuracy: 0.8462
Epoch 5/20
7/7 [==============================] - 5s 639ms/step - loss: 0.4971 - accuracy: 0.7933 - val_loss: 0.4737 - val_accuracy: 0.7885
Epoch 6/20
7/7 [==============================] - 3s 458ms/step - loss: 0.3965 - accuracy: 0.8510 - val_loss: 0.3678 - val_accuracy: 0.8846
Epoch 7/20
7/7 [==============================] - 3s 468ms/step - loss: 0.3479 - accuracy: 0.8702 - val_loss: 0.3478 - val_accuracy: 0.8654
Epoch 8/20
7/7 [======

In [14]:
# BiRNN model using GloVe embeddings - PREPROCESSED DATA - WITH DROPOUT
birnn_glove_prep_drop = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size_prep, embedding_dim, input_length=X_train_glove_prep.shape[1], weights=[embedding_matrix_prep], trainable=False),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim, return_sequences=True)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
birnn_glove_prep_drop.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=3e-4), metrics=['accuracy'])
birnn_glove_prep_drop.fit(X_train_glove_prep, y_train_glove_prep, epochs=20, validation_data=(X_test_glove_prep, y_test_glove_prep))

Epoch 1/20
7/7 [==============================] - 17s 1s/step - loss: 0.6957 - accuracy: 0.5240 - val_loss: 0.6859 - val_accuracy: 0.6538
Epoch 2/20
7/7 [==============================] - 3s 474ms/step - loss: 0.6792 - accuracy: 0.6490 - val_loss: 0.6749 - val_accuracy: 0.7115
Epoch 3/20
7/7 [==============================] - 3s 477ms/step - loss: 0.6667 - accuracy: 0.6731 - val_loss: 0.6597 - val_accuracy: 0.6923
Epoch 4/20
7/7 [==============================] - 6s 849ms/step - loss: 0.6380 - accuracy: 0.7163 - val_loss: 0.6338 - val_accuracy: 0.7308
Epoch 5/20
7/7 [==============================] - 4s 593ms/step - loss: 0.6055 - accuracy: 0.7644 - val_loss: 0.5934 - val_accuracy: 0.7692
Epoch 6/20
7/7 [==============================] - 3s 485ms/step - loss: 0.5428 - accuracy: 0.7885 - val_loss: 0.5164 - val_accuracy: 0.7885
Epoch 7/20
7/7 [==============================] - 4s 636ms/step - loss: 0.4816 - accuracy: 0.7981 - val_loss: 0.4924 - val_accuracy: 0.7885
Epoch 8/20
7/7 [======

In [15]:
# Evaluate TF-IDF models
tfidf_raw = birnn_tfidf_raw.evaluate(X_test_tfidf_raw, y_test_tfidf_raw)
tfidf_raw_drop = birnn_tfidf_raw_drop.evaluate(X_test_tfidf_raw, y_test_tfidf_raw)
tfidf_prep = birnn_tfidf_prep.evaluate(X_test_tfidf_prep, y_test_tfidf_prep)
tfidf_prep_drop = birnn_tfidf_prep.evaluate(X_test_tfidf_prep, y_test_tfidf_prep)

# Evaluate GloVe models
glove_raw = birnn_glove_raw.evaluate(X_test_glove_raw, y_test_glove_raw)
glove_raw_drop = birnn_glove_raw_drop.evaluate(X_test_glove_raw, y_test_glove_raw)
glove_prep = birnn_glove_prep.evaluate(X_test_glove_prep, y_test_glove_prep)
glove_prep_drop = birnn_glove_prep_drop.evaluate(X_test_glove_prep, y_test_glove_prep)

2/2 [==============================] - 0s 100ms/step - loss: 0.2473 - accuracy: 0.9231


In [20]:
# Print TF-IDF Model Accuracies
print("TF-IDF Model Accuracies:")
print("tfidf_raw Accuracy: %.2f%%" % (tfidf_raw[1]*100))
print("tfidf_raw_drop Accuracy: %.2f%%" % (tfidf_raw_drop[1]*100))
print("tfidf_prep Accuracy: %.2f%%" % (tfidf_prep[1]*100))
print("tfidf_prep_drop Accuracy: %.2f%%" % (tfidf_prep_drop[1]*100))

# Print GloVe Model Accuracies
print("\nGloVe Model Accuracies:")
print("glove_raw Accuracy: %.2f%%" % (glove_raw[1]*100))
print("glove_raw_drop Accuracy: %.2f%%" % (glove_raw_drop[1]*100))
print("glove_prep Accuracy: %.2f%%" % (glove_prep[1]*100))
print("glove_prep_drop Accuracy: %.2f%%" % (glove_prep_drop[1]*100))

TF-IDF Model Accuracies:
tfidf_raw Accuracy: 90.38%
tfidf_raw_drop Accuracy: 88.46%
tfidf_prep Accuracy: 88.46%
tfidf_prep_drop Accuracy: 88.46%

GloVe Model Accuracies:
glove_raw Accuracy: 86.54%
glove_raw_drop Accuracy: 76.92%
glove_prep Accuracy: 80.77%
glove_prep_drop Accuracy: 92.31%


In [17]:
# Saving TF-IDF Models as h5 files
birnn_tfidf_raw.save('/content/models/birnn_tfidf_raw.h5')
birnn_tfidf_raw_drop.save('/content/models/birnn_tfidf_raw_drop.h5')
birnn_tfidf_prep.save('/content/models/birnn_tfidf_prep.h5')
birnn_tfidf_prep_drop.save('/content/models/birnn_tfidf_prep_drop.h5')

# Saving GloVe Models as h5 files
birnn_glove_raw.save('/content/models/birnn_glove_raw.h5')
birnn_glove_raw_drop.save('/content/models/birnn_glove_raw_drop.h5')
birnn_glove_prep.save('/content/models/birnn_glove_prep.h5')
birnn_glove_prep_drop.save('/content/models/birnn_glove_prep_drop.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
# Saving TF-IDF Models as Pickle files
with open("/content/models/birnn_tfidf_raw.pkl", 'wb') as file:
    pickle.dump(birnn_tfidf_raw, file)
with open("/content/models/birnn_tfidf_raw_drop.pkl", 'wb') as file:
    pickle.dump(birnn_tfidf_raw_drop, file)
with open("/content/models/birnn_tfidf_prep.pkl", 'wb') as file:
    pickle.dump(birnn_tfidf_prep, file)
with open("/content/models/birnn_tfidf_prep_drop.pkl", 'wb') as file:
    pickle.dump(birnn_tfidf_prep_drop, file)

# Saving GloVe Models as Pickle files
with open("/content/models/birnn_glove_raw.pkl", 'wb') as file:
    pickle.dump(birnn_glove_raw, file)
with open("/content/models/birnn_glove_raw_drop.pkl", 'wb') as file:
    pickle.dump(birnn_glove_raw_drop, file)
with open("/content/models/birnn_glove_prep.pkl", 'wb') as file:
    pickle.dump(birnn_glove_prep, file)
with open("/content/models/birnn_glove_prep_drop.pkl", 'wb') as file:
    pickle.dump(birnn_glove_prep_drop, file)

In [19]:
!zip -r /content/models.zip /content/models

updating: content/models/ (stored 0%)
updating: content/models/birnn_tfidf_prep_drop.h5 (deflated 15%)
updating: content/models/birnn_glove_prep.pkl (deflated 9%)
updating: content/models/birnn_tfidf_prep_drop.pkl (deflated 14%)
updating: content/models/birnn_tfidf_prep.h5 (deflated 15%)
updating: content/models/birnn_tfidf_raw_drop.pkl (deflated 15%)
updating: content/models/birnn_glove_prep.h5 (deflated 9%)
updating: content/models/birnn_glove_prep_drop.pkl (deflated 9%)
updating: content/models/birnn_glove_raw.h5 (deflated 9%)
updating: content/models/birnn_tfidf_prep.pkl (deflated 14%)
updating: content/models/birnn_glove_raw_drop.h5 (deflated 9%)
updating: content/models/birnn_glove_raw_drop.pkl (deflated 9%)
updating: content/models/birnn_tfidf_raw_drop.h5 (deflated 15%)
updating: content/models/birnn_glove_prep_drop.h5 (deflated 9%)
updating: content/models/birnn_tfidf_raw.h5 (deflated 15%)
updating: content/models/birnn_glove_raw.pkl (deflated 9%)
updating: content/models/birnn